# Here's what I did
<pre>
- Overall EDA of the data:
    1. Shapes:                         data_shape.txt
    2. Missing count:                  column_missing_count.csv
    3. Types:                          column_type.csv
    4. Shared columns across data:     columns_shared_across_csvs.json
    5. Terminology for columns:        column_description.csv

- Result: ./eda/overall/artifact/*
</pre>

In [1]:
import pandas as pd
import glob
import json
import os
import re

In [2]:
ARTIFACT_DIR = r"../../artifact/eda/pandas"
DOWNLOAD_PATH = r"../../downloads/olist"

In [3]:
os.makedirs(ARTIFACT_DIR, exist_ok=True)

In [4]:
src_path = os.path.join(DOWNLOAD_PATH, '*')
csv_paths = glob.glob(src_path)
csv_paths.sort()

In [5]:
abs_csv_paths = list(map(os.path.abspath, csv_paths))

In [6]:
config_json = {}

file_names = []
for abs_path in abs_csv_paths:
    _path = os.path.splitext(abs_path)[0]
    file_name = os.path.split(_path)[1]
    file_names.append(file_name)

processed_name = [
    re.sub(r"^olist_(.*)_dataset$", r"\1", name)
    for name in file_names
]
dataset_var = dict(zip(processed_name, abs_csv_paths))
with open(os.path.join(ARTIFACT_DIR, 'paths.json'), "w") as f:
    json.dump(dataset_var, f, indent=4)

## Shape of data

In [7]:
df_by_path = {}
for csv_path in csv_paths:
    df = pd.read_csv(csv_path)
    df_by_path[csv_path] = df
    
save_path = os.path.join(ARTIFACT_DIR, "data_shape.txt")
with open(save_path, 'w') as f:
    for csv_path, df in df_by_path.items():
        file_name = os.path.split(csv_path)[-1]
        f.write(f"{file_name:<40} -> {df.shape}\n")

## Check missing count

In [8]:
missing_df = pd.DataFrame(columns=['file_name', 'column_name', 'missing_count'])
for csv_path, df in df_by_path.items():
    check_missing_ds = df.isna().sum()
    tmp_df = check_missing_ds.to_frame(name='missing_count')
    tmp_df.index.name = 'column_name'
    tmp_df = tmp_df.reset_index()
    tmp_df['file_name'] = os.path.split(csv_path)[-1]
    missing_df = pd.concat([missing_df, tmp_df])
    save_path = os.path.join(ARTIFACT_DIR, "column_missing_count.csv")
    missing_df.to_csv(save_path, index=False)

## All type of columns

In [9]:
type_df = pd.DataFrame(columns=['file_name', 'column_name', 'type'])
for csv_path, df in df_by_path.items():
    type_ds = df.dtypes
    tmp_df = type_ds.to_frame(name='type')
    tmp_df.index.name = 'column_name'
    tmp_df = tmp_df.reset_index()
    tmp_df['file_name'] = os.path.split(csv_path)[-1]
    type_df = pd.concat([type_df, tmp_df])
    save_path = os.path.join(ARTIFACT_DIR, "column_type.csv")

    type_df.to_csv(save_path, index=False)

## Columns shared across data

In [10]:
columns_by_csv = {}
for csv_path, df in df_by_path.items():
    columns = sorted(df.columns.to_list())
    columns_by_csv[csv_path] = columns

# print(json.dumps(columns_by_csv, indent=4))

In [11]:
all_unique_columns = []
for columns in list(columns_by_csv.values()):
    all_unique_columns += columns
all_unique_columns = sorted(set(all_unique_columns))

In [12]:
column_by_csvs = {}
for unique_col in all_unique_columns:
    column_by_csvs[unique_col] = []

for csv_path, cols in columns_by_csv.items():
    for unique_col in column_by_csvs.keys():
        if unique_col in cols:
            file_name = os.path.split(csv_path)[-1]
            column_by_csvs[unique_col].append(file_name)
            
columns_shared_across_csvs = dict(filter(lambda item: len(item[1]) > 1, column_by_csvs.items()))
save_path = os.path.join(ARTIFACT_DIR, "columns_shared_across_csvs.json")
with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(columns_shared_across_csvs, f, indent=4, ensure_ascii=False)

## Column description

In [13]:
def generate_column_description(description_data, output_file):    
    description_list = []
    for line in description_data.strip().split('\n')[1:]:
        file_name, column_name, description = line.split(',', 2)
        description_list.append([file_name, column_name, description])

    df_description = pd.DataFrame(description_list, columns=['file_name', 'column_name', 'description'])
    df_description.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Done: '{output_file}'")


description_data = \
"""file_name,column_name,description
olist_customers_dataset.csv,customer_id,고객을 식별하기 위한 고유 ID
olist_customers_dataset.csv,customer_unique_id,고객의 고유 식별자(중복되지 않는 ID)
olist_customers_dataset.csv,customer_zip_code_prefix,고객의 우편번호 접두사
olist_customers_dataset.csv,customer_city,고객이 거주하는 도시
olist_customers_dataset.csv,customer_state,고객이 거주하는 주(state)
olist_geolocation_dataset.csv,geolocation_zip_code_prefix,위치 정보에 해당하는 우편번호 접두사
olist_geolocation_dataset.csv,geolocation_lat,위도(latitude)
olist_geolocation_dataset.csv,geolocation_lng,경도(longitude)
olist_geolocation_dataset.csv,geolocation_city,위치 정보에 해당하는 도시
olist_geolocation_dataset.csv,geolocation_state,위치 정보에 해당하는 주(state)
olist_order_items_dataset.csv,order_id,주문을 식별하기 위한 고유 ID
olist_order_items_dataset.csv,order_item_id, 해당 주문 내에서 각 상품 아이템을 구분하는 번호
olist_order_items_dataset.csv,product_id,상품을 식별하기 위한 고유 ID
olist_order_items_dataset.csv,seller_id,판매자를 식별하기 위한 고유 ID
olist_order_items_dataset.csv,shipping_limit_date,판매자가 상품을 배송사에 전달해야 하는 기한
olist_order_items_dataset.csv,price,상품 가격
olist_order_items_dataset.csv,freight_value,배송비(운송 비용)
olist_order_payments_dataset.csv,order_id,주문을 식별하기 위한 고유 ID
olist_order_payments_dataset.csv,payment_sequential,결제 순서(결제와 관련된 순차적 번호)
olist_order_payments_dataset.csv,payment_type,결제 유형(예: credit_card, boleto 등)
olist_order_payments_dataset.csv,payment_installments,할부 횟수
olist_order_payments_dataset.csv,payment_value,결제 금액
olist_order_reviews_dataset.csv,review_id,리뷰를 식별하기 위한 고유 ID
olist_order_reviews_dataset.csv,order_id,주문을 식별하기 위한 고유 ID
olist_order_reviews_dataset.csv,review_score,리뷰 점수(예: 1~5점)
olist_order_reviews_dataset.csv,review_comment_title,리뷰 제목
olist_order_reviews_dataset.csv,review_comment_message,리뷰 코멘트 내용
olist_order_reviews_dataset.csv,review_creation_date,리뷰가 작성된 날짜
olist_order_reviews_dataset.csv,review_answer_timestamp,리뷰에 대한 답변이 작성된 날짜 및 시간
olist_orders_dataset.csv,order_id,주문을 식별하기 위한 고유 ID
olist_orders_dataset.csv,customer_id,고객을 식별하기 위한 고유 ID
olist_orders_dataset.csv,order_status,주문 상태(예: delivered, shipped, canceled 등)
olist_orders_dataset.csv,order_purchase_timestamp,주문이 생성된 날짜 및 시간
olist_orders_dataset.csv,order_approved_at,주문이 승인된 날짜 및 시간
olist_orders_dataset.csv,order_delivered_carrier_date,주문이 배송사에 전달된 날짜
olist_orders_dataset.csv,order_delivered_customer_date,주문이 고객에게 배송 완료된 날짜
olist_orders_dataset.csv,order_estimated_delivery_date,주문의 예상 배송 날짜
olist_products_dataset.csv,product_id,상품을 식별하기 위한 고유 ID
olist_products_dataset.csv,product_category_name,상품 카테고리 이름(원어)
olist_products_dataset.csv,product_name_lenght,상품 이름의 길이(문자 수)
olist_products_dataset.csv,product_description_lenght,상품 설명의 길이(문자 수)
olist_products_dataset.csv,product_photos_qty,상품 사진 개수
olist_products_dataset.csv,product_weight_g,상품 무게(g)
olist_products_dataset.csv,product_length_cm,상품 길이(cm)
olist_products_dataset.csv,product_height_cm,상품 높이(cm)
olist_products_dataset.csv,product_width_cm,상품 너비(cm)
olist_sellers_dataset.csv,seller_id,판매자를 식별하기 위한 고유 ID
olist_sellers_dataset.csv,seller_zip_code_prefix,판매자의 우편번호 접두사
olist_sellers_dataset.csv,seller_city,판매자가 위치한 도시
olist_sellers_dataset.csv,seller_state,판매자가 위치한 주(state)
product_category_name_translation.csv,product_category_name,상품 카테고리 이름(원어)
product_category_name_translation.csv,product_category_name_english,상품 카테고리 이름(영어)
"""

output_file = os.path.join(ARTIFACT_DIR, "column_description.csv")
generate_column_description(description_data, output_file)

Done: '../../artifact/eda/pandas/column_description.csv'
